# AI Working Group
#### 9/28/18

### Information Value (IV) and Weights of Evidence (WoE)

We'll look at techniques involving IV and WoE that can potentially assist the implementation of the MRM AI WG Idea "Variable Selection for PPNR Models" by Earvin.

https://support.sas.com/resources/papers/proceedings15/3242-2015.pdf

https://www.lexjansen.com/sesug/2014/SD-20.pdf

https://medium.com/@sundarstyles89/weight-of-evidence-and-information-value-using-python-6f05072e83eb

https://www.kaggle.com/puremath86/iv-woe-starter-for-python/notebook

https://www.kaggle.com/varungitboi/employee-salary-dataset

https://ramhiser.com/2012/11/23/how-to-download-kaggle-data-with-python-and-requests-dot-py/

Pandas:
https://pandas.pydata.org/pandas-docs/stable/indexing.html


IV in Python:
https://www.kaggle.com/puremath86/iv-woe-starter-for-python


In [1]:
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

In [2]:
data = pd.read_csv('180928_employee_data.csv')
data.head()

,Unnamed: 0,id,groups,age,healthy_eating,active_lifestyle,salary
0,0,0,A,36,5,5,2297
1,1,1,A,55,3,5,1134
2,2,2,A,61,8,1,4969
3,3,3,O,29,3,6,902
4,4,4,O,34,6,2,3574


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
Unnamed: 0          1000 non-null int64
id                  1000 non-null int64
groups              1000 non-null object
age                 1000 non-null int64
healthy_eating      1000 non-null int64
active_lifestyle    1000 non-null int64
salary              1000 non-null int64
dtypes: int64(6), object(1)
memory usage: 54.8+ KB


In [4]:
data.describe()

,Unnamed: 0,id,age,healthy_eating,active_lifestyle,salary
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
mean,499.500000,499.500000,41.155000,4.944000,5.683000,2227.46100
std,288.819436,288.819436,13.462995,2.013186,2.048587,1080.20976
min,0.000000,0.000000,18.000000,0.000000,0.000000,553.00000
25%,249.750000,249.750000,30.000000,4.000000,4.000000,1360.00000
50%,499.500000,499.500000,41.000000,5.000000,6.000000,2174.00000
75%,749.250000,749.250000,53.000000,6.000000,7.000000,2993.75000
max,999.000000,999.000000,64.000000,10.000000,10.000000,5550.00000


In [5]:
bins = [0,data['salary'].mean(),data['salary'].max()]
labels = ["0","1"]
data['bin_class'] = pd.cut(data['salary'],bins=bins,labels=labels)
data.head()

,Unnamed: 0,id,groups,age,healthy_eating,active_lifestyle,salary,bin_class
0,0,0,A,36,5,5,2297,1
1,1,1,A,55,3,5,1134,0
2,2,2,A,61,8,1,4969,1
3,3,3,O,29,3,6,902,0
4,4,4,O,34,6,2,3574,1


In [6]:
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

In [7]:
def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

In [8]:
def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv) 

In [9]:
final_iv, iv = data_vars(data,data.bin_class)

KeyboardInterrupt: 

In [10]:
# Calculate information value
def calc_iv(df, feature, target, pr=False):
    """
    Set pr=True to enable printing of output.
    
    Output: 
      * iv: float,
      * data: pandas.DataFrame
    """

    lst = []

    df[feature] = df[feature].fillna("NULL")

    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature,                                                        # Variable
                    val,                                                            # Value
                    df[df[feature] == val].count()[feature],                        # All
                    df[(df[feature] == val) & (df[target] == 0)].count()[feature],  # Good (think: Fraud == 0)
                    df[(df[feature] == val) & (df[target] == 1)].count()[feature]]) # Bad (think: Fraud == 1)

    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Good', 'Bad'])

    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])

    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})

    data['IV'] = data['WoE'] * (data['Distribution Good'] - data['Distribution Bad'])

    data = data.sort_values(by=['Variable', 'Value'], ascending=[True, True])
    data.index = range(len(data.index))

    if pr:
        print(data)
        print('IV = ', data['IV'].sum())


    iv = data['IV'].sum()
    # print(iv)

    return iv, data

In [20]:
iv, dt = calc_iv(data,'active_lifestyle','bin_class',pr=True)

            Variable  Value  All  Good  Bad  Share  Bad Rate  \
0   active_lifestyle      0    7     0    7  0.007  1.000000   
1   active_lifestyle      1   26     4   22  0.026  0.846154   
2   active_lifestyle      2   34     8   26  0.034  0.764706   
3   active_lifestyle      3   92    39   53  0.092  0.576087   
4   active_lifestyle      4  104    51   53  0.104  0.509615   
5   active_lifestyle      5  168    75   93  0.168  0.553571   
6   active_lifestyle      6  213   116   97  0.213  0.455399   
7   active_lifestyle      7  163    99   64  0.163  0.392638   
8   active_lifestyle      8  114    82   32  0.114  0.280702   
9   active_lifestyle      9   64    53   11  0.064  0.171875   
10  active_lifestyle     10   15    10    5  0.015  0.333333   

    Distribution Good  Distribution Bad       WoE        IV  
0            0.000000          0.015119  0.000000 -0.000000  
1            0.007449          0.047516 -1.853019  0.074246  
2            0.014898          0.056156 -1.32

/opt/conda/envs/py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log


nan

In [17]:
data['bin_class'] = 0
data.describe()

,Unnamed: 0,id,age,healthy_eating,active_lifestyle,salary,bin_class
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.0
mean,499.500000,499.500000,41.155000,4.944000,5.683000,2227.46100,0.0
std,288.819436,288.819436,13.462995,2.013186,2.048587,1080.20976,0.0
min,0.000000,0.000000,18.000000,0.000000,0.000000,553.00000,0.0
25%,249.750000,249.750000,30.000000,4.000000,4.000000,1360.00000,0.0
50%,499.500000,499.500000,41.000000,5.000000,6.000000,2174.00000,0.0
75%,749.250000,749.250000,53.000000,6.000000,7.000000,2993.75000,0.0
max,999.000000,999.000000,64.000000,10.000000,10.000000,5550.00000,0.0


In [18]:
data.loc[data['salary']>data['salary'].mean(),'bin_class'] = 1
data.describe()

,Unnamed: 0,id,age,healthy_eating,active_lifestyle,salary,bin_class
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000
mean,499.500000,499.500000,41.155000,4.944000,5.683000,2227.46100,0.463000
std,288.819436,288.819436,13.462995,2.013186,2.048587,1080.20976,0.498879
min,0.000000,0.000000,18.000000,0.000000,0.000000,553.00000,0.000000
25%,249.750000,249.750000,30.000000,4.000000,4.000000,1360.00000,0.000000
50%,499.500000,499.500000,41.000000,5.000000,6.000000,2174.00000,0.000000
75%,749.250000,749.250000,53.000000,6.000000,7.000000,2993.75000,1.000000
max,999.000000,999.000000,64.000000,10.000000,10.000000,5550.00000,1.000000


In [57]:
data['salary'].max()

5550